<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/project3_3113_zelalae2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To remove punctuation and numbers
import re
import pandas as pd
import numpy as np
import warnings

#Vis
import matplotlib.pyplot as plt
import seaborn as sns

#ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
import statsmodels.api as sm

# Import BeautifulSoup for Removing HTML Markup
from bs4 import BeautifulSoup
# A stop word list from the Python Natural Language Toolkit (NLTK)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize



In [2]:
# Suppress warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Goal
Your goal is to achieve an AUC score of at least 0.986 across all five test data splits.

## Resources:
- Kaggle: https://www.kaggle.com/c/word2vec-nlp-tutorial
- What we have tried 1: https://campuswire.com/c/GB46E5679/feed/785

In [4]:
DATA_DIR='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/Projects/Project3/Data/F24_Proj3_data'

##Libraries And Packages


In [5]:
# Removing HTML Markup: The BeautifulSoup Package
!pip install BeautifulSoup4


##**Load the Data**

In [22]:
train = pd.read_csv(f"{DATA_DIR}/split_5/train.csv") #quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

In [ ]:

print(train["review"][0])


Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce's short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty.


In [23]:
test = pd.read_csv(f"{DATA_DIR}/split_5/test.csv") #quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

In [ ]:
test

,id,review,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535,embedding_1536
0,598,"Now, Throw Momma from the Train was not a grea...",-0.013006,0.069379,-0.111694,0.005056,-0.043846,-0.076886,0.032729,0.033685,...,0.011099,0.024744,0.002237,0.040212,-0.003906,0.003135,0.011075,0.005938,-0.011499,-0.025222
1,12048,"( HR is what Himesh is called in the movie, I ...",-0.008011,0.045142,-0.015509,0.008454,0.010628,-0.030907,0.036003,0.021076,...,-0.001441,0.016769,0.004113,-0.013093,-0.018638,0.016478,-0.001745,0.008412,0.011929,-0.029799
2,40908,Everything is idyllic in Suburbia when the lit...,-0.021345,0.066069,-0.053891,0.034255,-0.007723,-0.007940,0.049389,0.032736,...,0.011167,0.001241,0.010713,-0.003246,0.003360,-0.014781,0.015568,0.008170,-0.012951,-0.057064
3,33283,A powerfully wonderful movie. You are held in ...,-0.021834,0.039822,-0.041830,0.063382,0.001213,-0.009429,0.034238,0.053831,...,-0.003172,-0.005201,-0.022103,0.029854,0.004157,-0.015735,0.001184,0.000577,0.024405,-0.003064
4,40600,"I remember reading all the horrible, horrible ...",-0.029196,0.077836,-0.038469,-0.009751,-0.021623,-0.028690,0.027791,0.016452,...,-0.000343,0.003688,-0.005005,-0.006681,0.000428,0.033017,-0.005223,0.014331,0.002518,-0.021595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,14480,Where Da Vinci code introduced us to Dr. Rober...,-0.041165,0.040130,-0.049036,-0.012007,-0.005579,-0.004045,-0.001575,0.031974,...,0.018706,-0.008356,-0.016777,-0.012117,-0.009049,0.010518,0.018990,0.000415,0.037256,-0.008596
24996,7634,"I haven't seen Ishtar, but I did have the misf...",-0.077530,0.062157,-0.046964,0.005665,-0.004714,-0.069126,0.058468,0.028106,...,-0.002066,0.020728,-0.019728,0.017230,0.008846,-0.016398,-0.025032,0.025301,0.022790,-0.041788
24997,3620,Peter Segal's 1995 commercial hit & now cult-c...,-0.008801,0.021090,-0.071936,0.021405,0.017185,-0.044679,0.001035,0.034280,...,0.027167,0.005666,-0.031939,-0.002144,-0.045309,-0.030094,0.023296,0.022418,0.073872,-0.003846
24998,24858,The reviewer in Variety said this was \overwri...,-0.044322,0.079591,-0.030821,0.024228,-0.032444,-0.018616,0.036316,-0.003181,...,0.010812,0.004124,0.029984,0.019636,-0.007051,0.007365,0.019545,0.009903,0.009910,-0.031240


In [8]:
# This should be remove before submission
test_y = pd.read_csv(f"{DATA_DIR}/split_4/test_y.csv") #quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

In [ ]:
test_y

,id,sentiment
0,598,1
1,12048,0
2,40908,1
3,33283,1
4,40600,0
...,...,...
24995,14480,1
24996,7634,0
24997,3620,1
24998,24858,0


##Data Cleaning and Text Preprocessing

##Removing HTML Markup: The BeautifulSoup Package

**Remove punctuation and numbers**

## Frequently occurring words that don't carry much meaning. Such words are called "stop words";
- import a stop word list from the Python Natural Language Toolkit (NLTK)

In [24]:

nltk.download('stopwords')  # Download text data sets, including stop words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text()
    #
    # 2. Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))
    #
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    #
    # 6. Join the words back into one string separated by space,
    # and return the result.

    return( " ".join( meaningful_words ))

# Loop through and clean all of the training set at once

Sometimes it can be annoying to wait for a lengthy piece of code to run. It can be helpful to write code so that it gives status updates. To have Python print a status update after every 1000 reviews that it processes, try adding a line or two to the code above:

In [26]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size
print("Cleaning and parsing the training set movie reviews...\n")
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [27]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000,
                             ngram_range=(1, 4),            # Use 1- to 4-grams
                             min_df=0.001,                       # Minimum term frequency
                             max_df=0.5,                      # Maximum document frequency
                             token_pattern=r"\b[\w+|']+\b") # Use word tokenizer: See Ethan's comment below

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [28]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names_out()
print(vocab)

['aaron' 'abandoned' 'abc' ... 'zombie' 'zombies' 'zone']


In [29]:

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

103 aaron
188 abandoned
99 abc
132 abilities
465 ability
1338 able
113 absence
106 absent
361 absolute
1507 absolutely
158 absolutely nothing
279 absurd
211 abuse
285 academy
122 academy award
465 accent
206 accents
302 accept
146 acceptable
157 accepted
349 accident
190 accidentally
117 accomplished
293 according
188 account
284 accurate
120 accused
185 achieve
150 achieved
117 achievement
1019 across
1234 act
125 act like
657 acted
6526 acting
216 acting bad
166 acting good
114 acting great
3242 action
131 action film
194 action movie
227 action scenes
162 action sequences
332 actions
2370 actor
4452 actors
115 actors actresses
1154 actress
356 actresses
359 acts
760 actual
4301 actually
174 ad
288 adam
432 adaptation
150 adapted
844 add
441 added
162 adding
305 addition
355 adds
106 adequate
122 admire
599 admit
121 admittedly
100 adorable
505 adult
363 adults
100 advance
159 advantage
476 adventure
194 adventures
271 advice
98 advise
345 affair
114 affected
109 afford
116 aforement

#Random Forest

In [34]:
print("Training the random forest...")

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100)

# Fit the forest to the training set, using the bag of words as
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


In [30]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = []

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()






Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [ ]:
# Use the random forest to make sentiment label predictions
result = forest.predict_proba(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "prob":result[:,1]} )

# Use pandas to write the comma-separated output file
output.to_csv(f"{DATA_DIR}/split_3/submission.csv", index=False, quoting=3 )

#Logistic Regression


In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score,make_scorer


# Separate features and target
X_train = train.drop(columns=['id', 'sentiment', 'review'])
y_train = train['sentiment']


logreg = LogisticRegression(
    solver='saga',  # 'saga' is the only solver supporting elastic net in sklearn
    max_iter=1000  # Increase if convergence issues occur
)

# Define a range of l1_ratio (alpha) values to experiment with
param_grid = {
    'penalty':['elasticnet'],
    'l1_ratio': [0],  # Elastic net mixing parameter (similar to alpha)
    'C': [10]  # Regularization strength, inverse of lambda in glmnet
}

# Define scoring as AUC for binary classification
scorer = make_scorer(roc_auc_score, needs_proba=True)

# Set up cross-validation with StratifiedKFold and AUC scoring
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=logreg,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    verbose=1,
    n_jobs=-1
)

# Perform grid search
grid_search.fit(train_data_features, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best AUC Score:", grid_search.best_score_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters: {'C': 10, 'l1_ratio': 0, 'penalty': 'elasticnet'}
Best AUC Score: 0.9361391506158239


In [32]:
# Train the final model with the best parameters
final_model = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    l1_ratio=grid_search.best_params_['l1_ratio'],
    C=grid_search.best_params_['C'],
    max_iter=1000
)

final_model.fit(X_train, y_train)

LogisticRegression(C=10, l1_ratio=0, max_iter=1000, penalty='elasticnet',
                   solver='saga')

In [33]:
# Making Prediction
X_test = test.drop(columns=['id', 'review'])

# Predict probabilities for the test data
predictions = final_model.predict_proba(X_test)[:, 1]  # Probabilities for positive sentiment


In [34]:
# Step 2: Save predictions to mysubmission1.txt
submission = pd.DataFrame({
    'id': test['id'],
    'prob': predictions
})

submission.to_csv(f'{DATA_DIR}/split_5/mysubmission.txt', index=False)


In [35]:
# Calculate AUC on the test data
y_test = pd.read_csv(f'{DATA_DIR}/split_5/test_y.csv')['sentiment']

test_auc_score = roc_auc_score(y_test, predictions)
print("Best AUC Score on Test Data:", test_auc_score)

Best AUC Score on Test Data: 0.9862843266582565


In [ ]:
elastic_net_cv = ElasticNetCV(alphas=[0.02],
                         l1_ratio= 0.1)

model= elastic_net_cv.fit(X_train_scaled, y_train)

    # Best alpha
best_alpha = elastic_net_cv.alpha_
print(f"Best alpha: {best_alpha}")